In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.preprocessing import PolynomialFeatures

import wrangle as w

import model as m
import env

import warnings
warnings.filterwarnings("ignore")

# Acquire/Prep

In [2]:
df = w.get_zillow_data()

In [3]:
df= w.zillow_prep(df)

In [4]:
train,validate,test = w.split(df)

In [5]:
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('loc_cluster', train,validate, test,['longitude', 'latitude','tax_value'],5)
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_price_size', train,validate, test,['taxamount', 'sqft', 'lot_sqft'],4)
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_delinquency_value', train,validate, test,['tax_value', 'sqft','lot_sqft','has_taxdelinquency'],4)


# Explore

# Model

In [6]:
features_to_model =['log_error','optional_features','has_taxdelinquency','cluster_price_size_0','cluster_price_size_1','cluster_delinquency_value_1','cluster_delinquency_value_2','age']

In [7]:
X_train_scaled,y_train, X_validate_scaled,y_validate, X_test_scaled, y_test = m.model_data_prep(train_scaled, validate_scaled,test_scaled, features_to_model)

In [21]:
metric1 = m.modeling(X_train_scaled, y_train, X_validate_scaled,y_validate, X_test_scaled, y_test)
metric1

RMSE for OLS using LinearRegression
Training/In-Sample:  0.17127170752416196 
Validation/Out-of-Sample:  0.1605080716857095
_______________
RMSE for Polynomial Model, degrees=2
Training/In-Sample:  0.17122849985206784 
Validation/Out-of-Sample:  0.1604951680775684


,model,RMSE_validate
0,mean_baseline,0.171387
1,OLS Regressor,0.160508
2,degree2,0.160495


In [22]:
metric1

,model,RMSE_validate
0,mean_baseline,0.171387
1,OLS Regressor,0.160508
2,degree2,0.160495


In [ ]:
# Example on how to run with diffirent features

In [18]:
features_to_model2 =['log_error','loc_cluster_0','longitude','latitude','age']

In [19]:
X_train_scaled2,y_train2, X_validate_scaled2,y_validate2, X_test_scaled2, y_test2 = m.model_data_prep(train_scaled, validate_scaled,test_scaled, features_to_model2)

In [23]:
metric2 = m.modeling(X_train_scaled2, y_train2, X_validate_scaled2,y_validate2, X_test_scaled2, y_test2)
metric2

RMSE for OLS using LinearRegression
Training/In-Sample:  0.17131407303238583 
Validation/Out-of-Sample:  0.16037439321339708
_______________
RMSE for Polynomial Model, degrees=2
Training/In-Sample:  0.17124862820966363 
Validation/Out-of-Sample:  0.16032494346533144


,model,RMSE_validate
0,mean_baseline,0.171387
1,OLS Regressor,0.160374
2,degree2,0.160325


In [25]:
metric=pd.concat([metric1,metric2])

In [26]:
metric

,model,RMSE_validate
0,mean_baseline,0.171387
1,OLS Regressor,0.160508
2,degree2,0.160495
0,mean_baseline,0.171387
1,OLS Regressor,0.160374
2,degree2,0.160325
